# SI 330: Data Manipulation 
## 06 - Enhancing pandas peformance

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

## Learning Objectives

* explain why we should be concerned with algorithmic efficiency
* be able to describe 5 types of Big O notation
* be able to report execution times in Jupyter
* be able to explain profiling output in Jupyter

**NOTE: You only need to conda install -y line_profiler ONCE!**

BPALKO

In [7]:
!conda install -y line_profiler

Solving environment: ...working... done

# All requested packages already installed.



## Big O Notation

* "order of growth"
* a way to categorize how algorithms behave with increasing data size
* commonly used in Computer Science

https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/

### $O(1)$

In [8]:
def IsFirstElementNull(aList):
    return aList[0] == None

### $O(n)$

In [9]:
def ContainsValue(aList, aValue):
    for e in aList:
        if e == aValue:
            return True
    return False

### $O(n^2$)

In [10]:
def ContainsDuplicates(aList):
    for i in range(len(aList)):
        for j in range(len(aList)):
            if i == j:
                continue
            if aList[i] == aList[j]:
                return True
    return False

### $O(2^n$)

In [11]:
def Fibonacci(n):
    if (n <= 1):
        return n
    return Fibonacci(n - 2) + Fibonacci(n -1 )

Recursion is expensive


### $O(log(n))$

In each iteration, we are halfway closer to finishing the problem.

https://stackoverflow.com/questions/2307283/what-does-olog-n-mean-exactly

This is why, for example, looking up people in a phone book is O(log n). You don't need to check every person in the phone book to find the right one; instead, you can simply divide-and-conquer by looking based on where their name is alphabetically, and in every section you only need to explore a subset of each section before you eventually find someone's phone number.

Of course, a bigger phone book will still take you a longer time, but it won't grow as quickly as the proportional increase in the additional size.

We can expand the phone book example to compare other kinds of operations and their running time. We will assume our phone book has businesses (the "Yellow Pages") which have unique names and people (the "White Pages") which may not have unique names. A phone number is assigned to at most one person or business. We will also assume that it takes constant time to flip to a specific page.

Here are the running times of some operations we might perform on the phone book, from best to worst:

O(1) (best case): Given the page that a business's name is on and the business name, find the phone number.

O(1) (average case): Given the page that a person's name is on and their name, find the phone number.

O(log n): Given a person's name, find the phone number by picking a random point about halfway through the part of the book you haven't searched yet, then checking to see whether the person's name is at that point. Then repeat the process about halfway through the part of the book where the person's name lies. (This is a binary search for a person's name.)

O(n): Find all people whose phone numbers contain the digit "5".

O(n): Given a phone number, find the person or business with that number.

O(n log n): There was a mix-up at the printer's office, and our phone book had all its pages inserted in a random order. Fix the ordering so that it's correct by looking at the first name on each page and then putting that page in the appropriate spot in a new, empty phone book.

For the below examples, we're now at the printer's office. Phone books are waiting to be mailed to each resident or business, and there's a sticker on each phone book identifying where it should be mailed to. Every person or business gets one phone book.

O(n log n): We want to personalize the phone book, so we're going to find each person or business's name in their designated copy, then circle their name in the book and write a short thank-you note for their patronage.

O(n2): A mistake occurred at the office, and every entry in each of the phone books has an extra "0" at the end of the phone number. Take some white-out and remove each zero.

O(n · n!): We're ready to load the phonebooks onto the shipping dock. Unfortunately, the robot that was supposed to load the books has gone haywire: it's putting the books onto the truck in a random order! Even worse, it loads all the books onto the truck, then checks to see if they're in the right order, and if not, it unloads them and starts over. (This is the dreaded bogo sort.)

O(nn): You fix the robot so that it's loading things correctly. The next day, one of your co-workers plays a prank on you and wires the loading dock robot to the automated printing systems. Every time the robot goes to load an original book, the factory printer makes a duplicate run of all the phonebooks! Fortunately, the robot's bug-detection systems are sophisticated enough that the robot doesn't try printing even more copies when it encounters a duplicate book for loading, but it still has to load every original and duplicate book that's been printed.

For more mathematical explanation you can checkout how the time complexity arrives to log n here. https://hackernoon.com/what-does-the-time-complexity-o-log-n-actually-mean-45f94bb5bfbf

## Timing and Profiling in Jupyter

The following code and examples are from [Sofia Heisler's repo](https://github.com/s-heisler/pycon2017-optimizing-pandas.git).
It is available at https://github.com/umsi-data-science/s-heisler-pycon2017-optimizing-pandas

In [12]:
import pandas as pd
import numpy as np
from math import *

### Read in the data

In [13]:
df = pd.read_csv('data/new_york_hotels.csv', encoding='cp1252')

In [14]:
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


In [15]:
df.columns

Index(['ean_hotel_id', 'name', 'address1', 'city', 'state_province',
       'postal_code', 'latitude', 'longitude', 'star_rating', 'high_rate',
       'low_rate'],
      dtype='object')

## Benchmarking example

#### Define the normalization function

In [29]:
def normalize(df, pd_series):
    pd_series = pd_series.astype(float)
    #print(pd_series)
    # Find upper and lower bound for outliers
    avg = np.mean(pd_series)
    sd  = np.std(pd_series)
    #print(avg,sd)
    lower_bound = avg - 2*sd
    upper_bound = avg + 2*sd

    # Collapse in the outliers
    df.loc[pd_series < lower_bound , "high_rate" ] = lower_bound
    df.loc[pd_series > upper_bound , "high_rate" ] = upper_bound
    # Finally, take the log (why add 1?)
    normalized_price = np.log(df["high_rate"].astype(float)+1)
    return normalized_price

In [30]:
normalize(df, df["high_rate"])

0       5.043601
1       5.193012
2       4.906533
3       5.734249
4       5.138090
5       5.247164
6       5.072872
7       5.422612
8       5.074459
9       4.941817
10      4.877789
11      5.688060
12      4.913116
13      5.247866
14      5.164786
15      4.895224
16      5.254627
17      4.266054
18      4.979557
19      5.193401
20      4.323205
21      4.262539
22      4.756517
23      4.941642
24      5.199712
25      5.274537
26      4.565084
27      5.248602
28      4.608963
29      4.204685
          ...   
1601    5.198497
1602    5.171733
1603    5.252430
1604    4.765246
1605    5.013697
1606    4.838581
1607    4.831349
1608    4.798102
1609    4.664397
1610    5.348915
1611    4.269558
1612    3.914272
1613    4.619369
1614    4.330733
1615    4.942531
1616    3.931826
1617    4.992433
1618    4.985728
1619    5.380391
1620    6.019323
1621    5.087535
1622    5.526272
1623    5.220356
1624    5.135974
1625    5.081404
1626    5.599754
1627    4.867534
1628    5.5227

#### Timing the normalization function

In [31]:
%timeit df["high_rate_normalized"] = normalize(df, df["high_rate"])

10.4 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Profiling the normalization function

In [32]:
    %load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [33]:
%lprun -f normalize df["high_rate_normalized"] = normalize(df, df["high_rate"])

Timer unit: 4.63768e-07 s

Total time: 0.0160784 s
File: <ipython-input-29-6e7cab35a1d5>
Function: normalize at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def normalize(df, pd_series):
     2         1        476.0    476.0      1.4      pd_series = pd_series.astype(float)
     3                                               #print(pd_series)
     4                                               # Find upper and lower bound for outliers
     5         1        603.0    603.0      1.7      avg = np.mean(pd_series)
     6         1        380.0    380.0      1.1      sd  = np.std(pd_series)
     7                                               #print(avg,sd)
     8         1          5.0      5.0      0.0      lower_bound = avg - 2*sd
     9         1          2.0      2.0      0.0      upper_bound = avg + 2*sd
    10                                           
    11                                               #

### <font color="magenta"> Q1 (2 points): Where is most of the time being spent?

Line 12 (16145.0) df.loc[pd_series < lower_bound , "high_rate" ] = lower_bound

### <font color="magenta"> Q2 (2 points): What order is the normalize function?  How can you figure this out?

O(1). Computationally look for implied for loop. Take samples to understand timings of the data further.

## There are at least 5 different ways to operate on a pandas Series:
* iterate with a for loop
* use iterrows
* use apply
* use vectorized functions
* use cython

## Haversine definition

https://en.wikipedia.org/wiki/Haversine_formula

In [34]:
def haversine(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

## Iterating using for and loc

### <font color="magenta"> Q3 (2 points): Write pythonic code that iterates over each row to calculate the distance from lat1= 40.671, lon1= -73.985 to each hotel's latitude and longitude.

In [40]:
%%timeit
for row in range(len(df)):
    row = df.loc[1]
    h = haversine(40.671, -73.985, row['latitude'], row['longitude'])
    


731 ms ± 36.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Iterrows Haversine

In [41]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985,\
                                      row['latitude'], row['longitude']))
df['distance'] = haversine_series

290 ms ± 66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Apply Haversine on rows

### Timing "apply"

In [42]:
%timeit df['distance'] =\
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

125 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Profiling "apply"

In [44]:
# Haversine applied on rows
%lprun -f haversine \
df.apply(lambda row: haversine(40.671, -73.985,\
                               row["latitude"], row["longitude"]), axis=1)

Timer unit: 4.63768e-07 s

Total time: 0.0558877 s
File: <ipython-input-34-ad000f5aaf6c>
Function: haversine at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def haversine(lat1, lon1, lat2, lon2):
     2      1631       3289.0      2.0      2.7      miles_constant = 3959
     3      1631      31819.0     19.5     26.4      lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
     4      1631       3478.0      2.1      2.9      dlat = lat2 - lat1 
     5      1631       2696.0      1.7      2.2      dlon = lon2 - lon1 
     6      1631      51829.0     31.8     43.0      a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
     7      1631      21043.0     12.9     17.5      c = 2 * np.arcsin(np.sqrt(a)) 
     8      1631       4021.0      2.5      3.3      mi = miles_constant * c
     9      1631       2333.0      1.4      1.9      return mi

## Vectorized implementation of Haversine applied on Pandas series

#### Timing vectorized implementation

In [45]:
# Vectorized implementation of Haversine applied on Pandas series
%timeit df['distance'] = haversine(40.671, -73.985,\
                                   df['latitude'], df['longitude'])

3.33 ms ± 46.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Profiling vectorized implementation

In [48]:
# Vectorized implementation profile
%lprun -f haversine haversine(40.671, -73.985,\
                              df["latitude"], df["longitude"])

Timer unit: 4.63768e-07 s

Total time: 0.00462701 s
File: <ipython-input-34-ad000f5aaf6c>
Function: haversine at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def haversine(lat1, lon1, lat2, lon2):
     2         1          5.0      5.0      0.1      miles_constant = 3959
     3         1        848.0    848.0      8.5      lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
     4         1        862.0    862.0      8.6      dlat = lat2 - lat1 
     5         1        657.0    657.0      6.6      dlon = lon2 - lon1 
     6         1       5639.0   5639.0     56.5      a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
     7         1       1340.0   1340.0     13.4      c = 2 * np.arcsin(np.sqrt(a)) 
     8         1        623.0    623.0      6.2      mi = miles_constant * c
     9         1          3.0      3.0      0.0      return mi

## Vectorized implementation of Haversine applied on NumPy arrays

#### Timing vectorized implementation

In [49]:
# Vectorized implementation of Haversine applied on NumPy arrays
%timeit df['distance'] = haversine(40.671, -73.985,\
                         df['latitude'].values, df['longitude'].values)

530 µs ± 109 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [50]:
%%timeit
# Convert pandas arrays to NumPy ndarrays
np_lat = df['latitude'].values
np_lon = df['longitude'].values

10.8 µs ± 87.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Profiling vectorized implementation

In [52]:
%lprun -f haversine df["distance"] = haversine(40.671, -73.985,\
                        df["latitude"].values, df["longitude"].values)

Timer unit: 4.63768e-07 s

Total time: 0.000916869 s
File: <ipython-input-34-ad000f5aaf6c>
Function: haversine at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def haversine(lat1, lon1, lat2, lon2):
     2         1          5.0      5.0      0.3      miles_constant = 3959
     3         1         94.0     94.0      4.8      lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
     4         1        330.0    330.0     16.7      dlat = lat2 - lat1 
     5         1         40.0     40.0      2.0      dlon = lon2 - lon1 
     6         1       1004.0   1004.0     50.8      a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
     7         1        488.0    488.0     24.7      c = 2 * np.arcsin(np.sqrt(a)) 
     8         1         14.0     14.0      0.7      mi = miles_constant * c
     9         1          2.0      2.0      0.1      return mi

### <font color="magenta"> Q4 (2 points): Record the times above and comment on how efficiency improves going from the first implementation to the last

731 ms, 290 ms, 125 ms, 3.33 ms, 530 micro, 10.8 micro. Efficiency is important once we get to large data sets improving speeds by factors of x10 greatly reduces time spent doing monotonous tasks. Terabytes could take minutes, therefore increasing costs!

## Not good enough?  Cythonize that loop

#### Load the cython extension

In [53]:
%load_ext cython

#### Run unaltered Haversine through Cython

In [54]:
%%cython -a

# Haversine cythonized (no other edits)
import numpy as np
cpdef haversine_cy(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

DistutilsPlatformError: Unable to find vcvarsall.bat

#### Time it

In [55]:
%timeit df['distance'] =\
       df.apply(lambda row: haversine_cy(40.671, -73.985,\
                row['latitude'], row['longitude']), axis=1)

NameError: ("name 'haversine_cy' is not defined", 'occurred at index 0')

WINDOWS WOULD NOT ALLOW ME TO RUN CYTHON

ERROR: DistutilsPlatformError: Unable to find vcvarsall.bat


### <font color="magenta"> Q5 (2 points): How much more efficient is the cython version compared to the best non-cython version from above?

Roughly the same, slightly less. (Was able to view data on neighbors machine)

#### Redefine Haversine with data types and C libraries

### FYI Only

In [ ]:
%%cython -a
# Haversine cythonized
from libc.math cimport sin, cos, acos, asin, sqrt

cdef deg2rad_cy(float deg):
    cdef float rad
    rad = 0.01745329252*deg
    return rad
    
cpdef haversine_cy_dtyped(float lat1, float lon1, float lat2, float lon2):
    cdef: 
        float dlon
        float dlat
        float a
        float c
        float mi
    
    lat1, lon1, lat2, lon2 = map(deg2rad_cy, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    mi = 3959 * c
    return mi


#### Time it

In [ ]:
%timeit df['distance'] =\
df.apply(lambda row: haversine_cy_dtyped(40.671, -73.985,\
                              row['latitude'], row['longitude']), axis=1)

# END OF NOTEBOOK
Please remember to submit your notebook in .ipynb and .html formats.